In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1
%env TF_FORCE_GPU_ALLOW_GROWTH=true

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1
env: TF_FORCE_GPU_ALLOW_GROWTH=true


In [2]:
import os
import pymatgen
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
structures = pd.read_pickle("datasets/structures_defects.pickle.gzip")

In [4]:
# TODO(inner perfectionist) eval is unsecure
defects = pd.read_csv(
  "datasets/dichalcogenides_innopolis_202105/descriptors.csv", index_col="_id",
  converters={"cell": eval, "defects": eval})

In [5]:
from megnet.models import MEGNetModel
from megnet.data.graph import GaussianDistance
from megnet.data.crystal import CrystalGraph
from megnet.utils.preprocessing import StandardScaler
from megnet.callbacks import ModelCheckpointMAE
from pymatgen.core import Lattice, Structure, Molecule

import tensorflow as tf
import numpy as np

In [6]:
import wandb
from wandb.keras import WandbCallback

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
structures = structures[structures.formation_energy > -10]
# TODO(kazeevn) pass structure as the global state
train, test = train_test_split(structures, test_size=0.25, random_state=42)

In [9]:
wandb.init(project='ai4material_design', entity='kazeev')

wandb: Currently logged in as: kazeev (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.11.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [10]:
config = wandb.config
config.target = "formation_energy"

In [11]:
from typing import Union, List, Dict
from megnet.data.graph import StructureGraphFixedRadius
from pymatgen.analysis.local_env import NearNeighbors
from megnet.data.graph import Converter
class ZAwareStrucutreGraph(StructureGraphFixedRadius):
    def __init__(
        self,
        nn_strategy: Union[str, NearNeighbors] = "MinimumDistanceNNAll",
        atom_converter: Converter = None,
        bond_converter: Converter = None,
        cutoff: float = 5.0,
    ):
      raise NotImplemented("Doesn't work due to difficult estate planning")
      self.cutoff = cutoff
      super().__init__(
              nn_strategy=nn_strategy, atom_converter=atom_converter, bond_converter=bond_converter, cutoff=self.cutoff
      )
        
      @staticmethod
      def get_atom_features(structure):
        return [[i.species.Z, i.coords[2]] for i in structure.sites]

In [12]:
nfeat_edge = 10
gc = CrystalGraph(bond_converter=GaussianDistance(np.linspace(0, 15, nfeat_edge), 0.5),
                  cutoff=15)
model = MEGNetModel(nfeat_edge=nfeat_edge, nfeat_global=2, graph_converter=gc, npass=2)

In [13]:
scaler = StandardScaler.from_training_data(train.defect_representation,
                                           train[config.target], is_intensive=False)
model.target_scaler = scaler

In [14]:
model.train(train.defect_representation, train[config.target],
            validation_structures=test.defect_representation,
            validation_targets=test[config.target],
            callbacks=[WandbCallback()],
            epochs=1000, verbose=1, patience=1000)

Epoch 1/1000


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:437: UserWarning:

Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model/set2set_atom/Reshape_9:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model/set2set_atom/Reshape_8:0", shape=(None,), dtype=float32), dense_shape=Tensor("gradient_tape/model/set2set_atom/Cast:0", shape=(1,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:437: UserWarning:

Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model/set2set_bond/Reshape_9:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model/set2set_bond/Reshape_8:0", shape=(None,), dtype=float32), dense_shape=Tensor("gradient_tape/model/set2set_bond/Cast:0", shape=(1,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of m

21/21 [==============================] - 18s 47ms/step - loss: 2.4278


INFO:megnet.callbacks:
Epoch 00001: val_mae improved from inf to 4.68139, saving model to callback/val_mae_00001_4.681393.hdf5


Epoch 2/1000
21/21 [==============================] - 1s 47ms/step - loss: 0.9712


...

Epoch 768/1000
21/21 [==============================] - 1s 50ms/step - loss: 2.1549e-04
Epoch 769/1000
21/21 [==============================] - 1s 49ms/step - loss: 2.4530e-04
Epoch 770/1000
21/21 [==============================] - 1s 49ms/step - loss: 2.3997e-04
Epoch 771/1000
21/21 [==============================] - 1s 49ms/step - loss: 2.9509e-04
Epoch 772/1000
21/21 [==============================] - 1s 49ms/step - loss: 2.9059e-04
Epoch 773/1000
21/21 [==============================] - 1s 49ms/step - loss: 3.2020e-04
Epoch 774/1000
21/21 [==============================] - 1s 49ms/step - loss: 4.4202e-04
Epoch 775/1000
21/21 [==============================] - 1s 49ms/step - loss: 3.0192e-04
Epoch 776/1000
21/21 [==============================] - 1s 49ms/step - loss: 3.1594e-04
Epoch 777/1000
21/21 [==============================] - 1s 49ms/step - loss: 2.5718e-04
Epoch 778/1000
21/21 [==============================] - 1s 49ms/step - loss: 2.1383e-04


INFO:megnet.callbacks:
Epoch 00778: val_mae improved from 0.10487 to 0.10138, saving model to callback/val_mae_00778_0.101379.hdf5


Epoch 779/1000
21/21 [==============================] - 1s 49ms/step - loss: 1.9870e-04
Epoch 780/1000
21/21 [==============================] - 1s 48ms/step - loss: 2.9449e-04
Epoch 781/1000
21/21 [==============================] - 1s 49ms/step - loss: 3.8912e-04
Epoch 782/1000
21/21 [==============================] - 1s 49ms/step - loss: 3.7662e-04
Epoch 783/1000
21/21 [==============================] - 1s 49ms/step - loss: 3.8484e-04
Epoch 784/1000
21/21 [==============================] - 1s 49ms/step - loss: 3.8198e-04
Epoch 785/1000
21/21 [==============================] - 1s 48ms/step - loss: 2.8846e-04
Epoch 786/1000
21/21 [==============================] - 1s 48ms/step - loss: 0.0013
Epoch 787/1000
21/21 [==============================] - 1s 49ms/step - loss: 0.0208
Epoch 788/1000
21/21 [==============================] - 1s 49ms/step - loss: 0.0072
Epoch 789/1000
21/21 [==============================] - 1s 49ms/step - loss: 0.0057
Epoch 790/1000
21/21 [==========================

21/21 [==============================] - 1s 49ms/step - loss: 5.2674e-04
Epoch 874/1000
21/21 [==============================] - 1s 49ms/step - loss: 5.7047e-04
Epoch 875/1000
21/21 [==============================] - 1s 48ms/step - loss: 5.6036e-04
Epoch 876/1000
21/21 [==============================] - 1s 48ms/step - loss: 6.3800e-04
Epoch 877/1000
21/21 [==============================] - 1s 50ms/step - loss: 5.9303e-04
Epoch 878/1000
21/21 [==============================] - 1s 49ms/step - loss: 5.0973e-04
Epoch 879/1000
21/21 [==============================] - 1s 49ms/step - loss: 4.5510e-04
Epoch 880/1000
21/21 [==============================] - 1s 49ms/step - loss: 5.0785e-04
Epoch 881/1000
21/21 [==============================] - 1s 49ms/step - loss: 4.3337e-04
Epoch 882/1000
21/21 [==============================] - 1s 49ms/step - loss: 4.9818e-04
Epoch 883/1000
21/21 [==============================] - 1s 49ms/step - loss: 4.7057e-04
Epoch 884/1000
21/21 [=========================

21/21 [==============================] - 1s 49ms/step - loss: 4.0149e-04
Epoch 967/1000
21/21 [==============================] - 1s 49ms/step - loss: 4.4079e-04
Epoch 968/1000
21/21 [==============================] - 1s 50ms/step - loss: 3.0622e-04
Epoch 969/1000
21/21 [==============================] - 1s 50ms/step - loss: 9.0605e-04
Epoch 970/1000
21/21 [==============================] - 1s 48ms/step - loss: 5.0534e-04
Epoch 971/1000
21/21 [==============================] - 1s 49ms/step - loss: 5.5608e-04
Epoch 972/1000
21/21 [==============================] - 1s 49ms/step - loss: 4.6601e-04
Epoch 973/1000
21/21 [==============================] - 1s 49ms/step - loss: 3.5631e-04
Epoch 974/1000
21/21 [==============================] - 1s 49ms/step - loss: 2.6201e-04
Epoch 975/1000
21/21 [==============================] - 1s 49ms/step - loss: 3.0040e-04
Epoch 976/1000
21/21 [==============================] - 1s 49ms/step - loss: 2.7937e-04
Epoch 977/1000
21/21 [=========================

In [15]:
model.save_model("models/MEGNet-defect-only/defect_E")

INFO:tensorflow:Assets written to: models/MEGNet-defect-only/defect_E/assets


INFO:tensorflow:Assets written to: models/MEGNet-defect-only/defect_E/assets
